# Mean Shift Algorithm Working

 The steps of the mean shift tracking algorithm are as follows:

Initialize the object location and size in the first frame of the video. Define a circular window around the object and compute its color histogram using a kernel function (such as Gaussian or Epanechnikov).

In the next frame, search for the window that has the most similar color histogram to the previous one, using a similarity measure (such as Bhattacharyya coefficient or histogram intersection). This can be done by iteratively shifting the window towards the mean of the pixels within it, until convergence or a maximum number of iterations is reached. This is called the mean shift procedure.

Update the object location and size based on the new window position and scale. Optionally, update the color histogram of the object as well.

Repeat steps 2 and 3 for each subsequent frame until the end of the video or until the object is lost.

In [ ]:
import numpy as np
import cv2

In [ ]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
face_cascade = cv2.CascadeClassifier(r'D:/CVR/haarcascade_frontalface_default.xml')
face_react = face_cascade.detectMultiScale(frame)
(face_x, face_y, w, h) = tuple(face_react[0])
track_window = (face_x, face_y, w, h)

roi = frame[face_y:face_y+h, face_x:face_x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_HSV2BGR)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret,frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        x,y, w, h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 5)
        cv2.imshow('back projection', dst)
        cv2.imshow('object tracking', img2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
        
cap.release()
cv2.destroyAllWindows()